# 英語と日本語の埋め込みベクトルの類似度を比較

In [27]:
import boto3
import json
import numpy as np

model_ids = [
    "amazon.titan-embed-text-v2:0",
    "amazon.titan-embed-image-v1",
    "amazon.titan-embed-text-v1",
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
]

def text_to_vector(text: str, model_id: str):
    # Bedrock Runtimeクライアントの作成
    runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
    
    # リクエストボディの準備
    
    # モデルの呼び出し
    if "cohere" in model_id:
        response = runtime.invoke_model(
            contentType="application/json",
            accept="application/json",
            modelId=model_id,
            body=json.dumps({
                "texts": [text],
                "input_type": "search_document"
            }).encode("utf-8")
        )
        # 埋め込みベクトルの抽出
        embedding = json.loads(response["body"].read())["embeddings"][0]
    else:
        response = runtime.invoke_model(
            contentType="application/json",
            accept="application/json",
            modelId=model_id,
            body=json.dumps({"inputText": text}).encode("utf-8")
        )
        # 埋め込みベクトルの抽出
        embedding = json.loads(response["body"].read())["embedding"]
    
    return embedding

words = ["りんご", "林檎", "apple", "🍎", "みかん", "軟骨", "なんこつ"]
embeddings = {}
for model_id in model_ids:
    for word in words:
        embeddings[f"{model_id}-{word}"] = text_to_vector(word, model_id)

In [31]:
# ベクトル間のコサイン類似度を計算
from scipy.spatial.distance import cosine
def cosine_similarity(v1, v2):
    return 1 - cosine(v1, v2)
print("vs りんご")
for model_id in model_ids:
    print(model_id)
    for word in words:
        print(f"{word} {cosine_similarity(embeddings[f"{model_id}-{word}"], embeddings[f"{model_id}-りんご"]):.3f}")
        #print(f"{word} {np.dot(embeddings[f"{model_id}-{word}"], embeddings[f"{model_id}-りんご"]):.3f}")
    print("")

vs りんご
amazon.titan-embed-text-v2:0
りんご 1.000
林檎 0.068
apple 0.078
🍎 0.038
みかん 0.258
軟骨 0.025
なんこつ 0.153

amazon.titan-embed-image-v1
りんご 1.000
林檎 0.847
apple 0.726
🍎 0.811
みかん 0.942
軟骨 0.777
なんこつ 0.934

amazon.titan-embed-text-v1
りんご 1.000
林檎 0.457
apple 0.185
🍎 0.323
みかん 0.733
軟骨 0.498
なんこつ 0.709

cohere.embed-english-v3
りんご 1.000
林檎 0.643
apple 0.443
🍎 0.561
みかん 0.795
軟骨 0.561
なんこつ 0.810

cohere.embed-multilingual-v3
りんご 1.000
林檎 0.843
apple 0.705
🍎 0.742
みかん 0.872
軟骨 0.765
なんこつ 0.830



個人的には「りんご」に対して「林檎」「apple」「🍎」に強い類似度が出てほしかったが、「なんこつ」「みかん」の方に強い類似度が出てしまった。